In [1]:
import pandas as pd
import numpy as np
import re
import warnings
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split

In [106]:
df = pd.read_csv("./dataset/train.csv")
df

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0
...,...,...,...
4995,4996,좋은 배우들로 3류영화를 찍은 안타까운 영화,0
4996,4997,진짜 드럽게 재미없다 에드워드 호퍼 그림에 배경 빼고는 볼게 아닌영화,0
4997,4998,가장 실망스러운 영화.. 지금까지 본영화중..,0
4998,4999,"이런 평점 테러, 네이버에서 좀 막아야 하는 것 아닌가?",1


In [107]:
df.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [108]:
df.drop('id',axis=1,inplace=True)

In [109]:
for i in df.document:
    if len(i)<10: 
        print(i)

In [110]:
train, val = train_test_split(df,random_state=0,test_size=0.2)
train.reset_index(inplace=True)
val.reset_index(inplace=True)

In [111]:
okt = Okt()

In [112]:
train['prep'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]"," ")
train['prep'] = train['prep'].str.replace(" +"," ")
train.head()

,index,document,label,prep
0,2913,우리 모두가 이들처럼 산다면 한국교회는 소망이 넘칠 것....반성합니다,1,우리 모두가 이들처럼 산다면 한국교회는 소망이 넘칠 것 반성합니다
1,3275,장백지를 좋아하게됐던 영화. 판타지애정물의 시초가 된 영화. 마이 슬퍼~,1,장백지를 좋아하게됐던 영화 판타지애정물의 시초가 된 영화 마이 슬퍼
2,775,정말 감탄사만 나온다.. 꼭 봐라 두번봐라!,1,정말 감탄사만 나온다 꼭 봐라 두번봐라
3,217,"이건 뭐야,책이 낫다... 이거 보지마셈",0,이건 뭐야 책이 낫다 이거 보지마셈
4,1245,끝도 허무하다.뻔한 결말을 왜 봐? 아-_-돈내나,0,끝도 허무하다 뻔한 결말을 왜 봐 아 돈내나


In [113]:
main_mor = []
stop_words = ['의','가','이','은','들','는','걍','과','들','걍','과','으로','도','을','를','으로','자','에','와','한','하다','에서','에게','셈','슴','거']
for sentence in train['prep']: 
    mor = okt.morphs(sentence, stem=True)
    token = [t for t in mor if t not in stop_words]
    tokenmor = " ".join(token)
    main_mor.append(tokenmor) 
train["main_mor"] = pd.DataFrame(main_mor)
#main_pos = 전처리 x 사용
train.head()

,index,document,label,prep,main_mor
0,2913,우리 모두가 이들처럼 산다면 한국교회는 소망이 넘칠 것....반성합니다,1,우리 모두가 이들처럼 산다면 한국교회는 소망이 넘칠 것 반성합니다,우리 모두 처럼 산다 면 한국 교회 소망 넘치다 것 반성
1,3275,장백지를 좋아하게됐던 영화. 판타지애정물의 시초가 된 영화. 마이 슬퍼~,1,장백지를 좋아하게됐던 영화 판타지애정물의 시초가 된 영화 마이 슬퍼,장백지 좋아하다 돼다 영화 판타지 애정 물의 시초 되다 영화 마이 슬프다
2,775,정말 감탄사만 나온다.. 꼭 봐라 두번봐라!,1,정말 감탄사만 나온다 꼭 봐라 두번봐라,정말 감탄사 만 나오다 꼭 보다 두 번 보다
3,217,"이건 뭐야,책이 낫다... 이거 보지마셈",0,이건 뭐야 책이 낫다 이거 보지마셈,이건 뭐 야 책 낫다 보다
4,1245,끝도 허무하다.뻔한 결말을 왜 봐? 아-_-돈내나,0,끝도 허무하다 뻔한 결말을 왜 봐 아 돈내나,끝 허무하다 뻔하다 결말 왜 보다 아 돈 내 나


In [114]:
'''
main_pos = [] 
for sentence in train['document']: 
    pos = okt.pos(sentence,stem=True)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str) 
train["main_pos"] = pd.DataFrame(main_pos)
#main_pos = 전처리 x 사용
train.head()
'''

'\nmain_pos = [] \nfor sentence in train[\'document\']: \n    pos = okt.pos(sentence,stem=True)\n    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]\n    main_words_str = " ".join(main_words)\n    main_pos.append(main_words_str) \ntrain["main_pos"] = pd.DataFrame(main_pos)\n#main_pos = 전처리 x 사용\ntrain.head()\n'

In [187]:
x_train = train.main_mor
y_train = train.label

# Tfidf

In [188]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=2,sublinear_tf=True,max_features=8700,ngram_range=(1, 3),analyzer="char")
x_train = vectorizer.fit_transform(x_train)
#x_val = vectorizer.transform(x_val)

In [117]:
x_train[1]

<1x9800 sparse matrix of type '<class 'numpy.float64'>'
	with 72 stored elements in Compressed Sparse Row format>

# VAl set 전처리

In [138]:
val['prep'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]"," ")
val['prep'] = val['prep'].str.replace(" +"," ")
val.head()

,index,document,label,prep,main_mor,main_pos
0,398,평점 왜이렇게낮나요? 너무사랑스러운 로맨스영화.. 재미있어요 ~~~,1,평점 왜이렇게낮나요 너무사랑스러운 로맨스영화 재미있어요,평점 왜 이렇게 낮다 너무 사랑스럽다 로맨스영화 재미있다,평점 왜 이렇게 낮다 너무 사랑스럽다 로맨스영화 재미있다
1,3833,예상 불가 대박 반전 최고최고,1,예상 불가 대박 반전 최고최고,예상 불가 대박 반전 최고 최고,예상 불가 대박 반전 최고 최고
2,4836,Ost가우선너무아름답고좋았구여내용도기가막혔습ㄴ다,1,가우선너무아름답고좋았구여내용도기가막혔습ㄴ다,가다 우선 너무 아름답다 좋다 내용 도기 막히다 습 ㄴ 다,가다 우선 너무 아름답다 좋다 내용 도기 막히다 습 다
3,4572,"긴박한 영화속 인물들, 하품과 한숨만 나오는 관객",0,긴박한 영화속 인물들 하품과 한숨만 나오는 관객,긴박하다 영화 속 인물 하품 한숨 만 나오다 관객,긴박하다 영화 속 인물 하품 한숨 나오다 관객
4,636,살짝 어두워지기 시작하는 부분.. 하지만 여전히 좋다!,1,살짝 어두워지기 시작하는 부분 하지만 여전히 좋다,살짝 어두워지다 시작 부분 하지만 여전하다 좋다,살짝 어두워지다 시작 하다 부분 여전하다 좋다


In [139]:
val_main_mor = []
for sentence in val['prep']: 
    mor = okt.morphs(sentence,stem=True)
    token = [t for t in mor if t not in stop_words]
    tokenmor = " ".join(token)
    val_main_mor.append(tokenmor) 
val["main_mor"] = pd.DataFrame(val_main_mor)
#main_pos = 전처리 x 사용
val.head()

,index,document,label,prep,main_mor,main_pos
0,398,평점 왜이렇게낮나요? 너무사랑스러운 로맨스영화.. 재미있어요 ~~~,1,평점 왜이렇게낮나요 너무사랑스러운 로맨스영화 재미있어요,평점 왜 이렇게 낮다 너무 사랑스럽다 로맨스영화 재미있다,평점 왜 이렇게 낮다 너무 사랑스럽다 로맨스영화 재미있다
1,3833,예상 불가 대박 반전 최고최고,1,예상 불가 대박 반전 최고최고,예상 불가 대박 반전 최고 최고,예상 불가 대박 반전 최고 최고
2,4836,Ost가우선너무아름답고좋았구여내용도기가막혔습ㄴ다,1,가우선너무아름답고좋았구여내용도기가막혔습ㄴ다,가다 우선 너무 아름답다 좋다 내용 도기 막히다 습 ㄴ 다,가다 우선 너무 아름답다 좋다 내용 도기 막히다 습 다
3,4572,"긴박한 영화속 인물들, 하품과 한숨만 나오는 관객",0,긴박한 영화속 인물들 하품과 한숨만 나오는 관객,긴박하다 영화 속 인물 하품 한숨 만 나오다 관객,긴박하다 영화 속 인물 하품 한숨 나오다 관객
4,636,살짝 어두워지기 시작하는 부분.. 하지만 여전히 좋다!,1,살짝 어두워지기 시작하는 부분 하지만 여전히 좋다,살짝 어두워지다 시작 부분 하지만 여전하다 좋다,살짝 어두워지다 시작 하다 부분 여전하다 좋다


In [120]:
val_main_pos = [] 
for sentence in val['document']: 
    pos = okt.pos(sentence,stem=True)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    val_main_pos.append(main_words_str) 
val["main_pos"] = pd.DataFrame(val_main_pos)
#main_pos = 전처리 x 사용
val.head()

,index,document,label,prep,main_mor,main_pos
0,398,평점 왜이렇게낮나요? 너무사랑스러운 로맨스영화.. 재미있어요 ~~~,1,평점 왜이렇게낮나요 너무사랑스러운 로맨스영화 재미있어요,평점 왜 이렇게 낮다 너무 사랑스럽다 로맨스영화 재미있다,평점 왜 이렇게 낮다 너무 사랑스럽다 로맨스영화 재미있다
1,3833,예상 불가 대박 반전 최고최고,1,예상 불가 대박 반전 최고최고,예상 불가 대박 반전 최고 최고,예상 불가 대박 반전 최고 최고
2,4836,Ost가우선너무아름답고좋았구여내용도기가막혔습ㄴ다,1,가우선너무아름답고좋았구여내용도기가막혔습ㄴ다,가다 우선 너무 아름답다 좋다 내용 도기 막히다 습 ㄴ 다,가다 우선 너무 아름답다 좋다 내용 도기 막히다 습 다
3,4572,"긴박한 영화속 인물들, 하품과 한숨만 나오는 관객",0,긴박한 영화속 인물들 하품과 한숨만 나오는 관객,긴박하다 영화 속 인물 하품 한숨 만 나오다 관객,긴박하다 영화 속 인물 하품 한숨 나오다 관객
4,636,살짝 어두워지기 시작하는 부분.. 하지만 여전히 좋다!,1,살짝 어두워지기 시작하는 부분 하지만 여전히 좋다,살짝 어두워지다 시작 부분 하지만 여전하다 좋다,살짝 어두워지다 시작 하다 부분 여전하다 좋다


In [189]:
x_val = val.main_mor
y_val = val.label

In [190]:
x_val = vectorizer.transform(x_val)

In [123]:
x_val

<1000x9800 sparse matrix of type '<class 'numpy.float64'>'
	with 50789 stored elements in Compressed Sparse Row format>

# MODEL

In [124]:
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix

In [125]:
def print_score(pred,y_val):
    acc = accuracy_score(y_val,pred)
    f1 = f1_score(y_val,pred)
    auc = roc_auc_score(y_val,pred)
    con = confusion_matrix(y_val,pred)
    
    print("acc : ",acc, "f1 : ",f1, "auc",auc)
    print(con)

In [126]:
from lightgbm import LGBMClassifier
lmodel = LGBMClassifier(max_depth=8,learning_rate=0.1,min_child_samples=5)
lmodel.fit(x_train,y_train)
pred = lmodel.predict(x_val)
print_score(pred,y_val)

acc :  0.823 f1 :  0.8065573770491803 auc 0.8208034628978099
[[454  73]
 [104 369]]


In [127]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(x_train,y_train)
pred = rf_model.predict(x_val)
print_score(pred,y_val)

acc :  0.815 f1 :  0.7986942328618063 auc 0.8129966983724541
[[448  79]
 [106 367]]


In [191]:
from sklearn.linear_model import LogisticRegression

lg_model = LogisticRegression(C=13,solver='liblinear')
lg_model.fit(x_train,y_train)
pred = lg_model.predict(x_val)
print(i," acc : ", round(accuracy_score(y_val,pred),3))

299  acc :  0.865


In [212]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier

nb_model = BernoulliNB()
nb_model.fit(x_train,y_train)
pred = nb_model.predict(x_val)
print_score(pred,y_val)


acc :  0.866 f1 :  0.8589473684210526 auc 0.8658247449562926
[[458  69]
 [ 65 408]]


In [216]:
sgd = SGDClassifier(random_state=1,loss='modified_huber')
sgd.fit(x_train,y_train)
pred = sgd.predict(x_val)
print(i,accuracy_score(y_val,pred))

99 0.856


In [217]:
from sklearn.ensemble import VotingClassifier

models = [('nb',nb_model), ('log',lg_model), ('sgd',sgd)]

hvc = VotingClassifier(estimators=models, voting='hard')
svc = VotingClassifier(estimators=models, voting='soft')
hvc.fit(x_train,y_train)
svc.fit(x_train,y_train)
print_score(hvc.predict(x_val),y_val)
print_score(svc.predict(x_val),y_val)

acc :  0.871 f1 :  0.8634920634920636 auc 0.8705685779733703
[[463  64]
 [ 65 408]]
acc :  0.865 f1 :  0.8577449947312962 auc 0.8647676625038613
[[458  69]
 [ 66 407]]


In [129]:
test = pd.read_csv("./dataset/test.csv")
test

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다
...,...,...
4995,4996,연계도 제대로 안되고 뭔가 부족한느낌이들고 찝찝한영화
4996,4997,내생에 최악의 영화중 하나 졸려서 미칠뻔....
4997,4998,지금까지 나온 드라마 중 최고였습니다. 100점 만점!
4998,4999,영화값으로 엄청난 공연을 보았다 - 이게 웬 횡재 ..


In [130]:
test['prep'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]"," ")
test['prep'] = test['prep'].str.replace(" +"," ")
test.head()
test_main_mor = []
for sentence in test['prep']: 
    mor = okt.morphs(sentence,stem=True)
    token = [t for t in mor if t not in stop_words]
    tokenmor = " ".join(token)
    test_main_mor.append(tokenmor) 
test["main_mor"] = pd.DataFrame(test_main_mor)
#main_pos = 전처리 x 사용
test.head()

,id,document,prep,main_mor
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함,시간 때우다 좋다 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱,훈훈하다 정이 느껴지다 영화 가족 끼리 드라마 보다 보다 딱
2,3,Childhood fantasy,,
3,4,멋있는 영화입니다. 잊을 수 없는!,멋있는 영화입니다 잊을 수 없는,멋있다 영화 이다 잊다 수 없다
4,5,너무 감동적이네요 펑펑 울었습니다,너무 감동적이네요 펑펑 울었습니다,너무 감동 적다 펑펑 울다


In [131]:
test_main_pos = [] 
for sentence in test['document']: 
    pos = okt.pos(sentence,stem=True)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    test_main_pos.append(main_words_str) 
test["document"] = pd.DataFrame(test_main_pos)
#main_pos = 전처리 x 사용
test.head()

,id,document,prep,main_mor
0,1,시간 때우다 좋다 영화 지루함,시간 때우기 좋은 영화 지루함,시간 때우다 좋다 영화 지루함
1,2,훈훈하다 정이 느껴지다 영화 가족 끼리 드라마 보다 보다 딱,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱,훈훈하다 정이 느껴지다 영화 가족 끼리 드라마 보다 보다 딱
2,3,,,
3,4,멋있다 영화 이다 잊다 수 없다,멋있는 영화입니다 잊을 수 없는,멋있다 영화 이다 잊다 수 없다
4,5,너무 감동 적다 펑펑 울다,너무 감동적이네요 펑펑 울었습니다,너무 감동 적다 펑펑 울다


In [218]:
X_test = test.main_mor
X_test_vec = vectorizer.transform(X_test)

In [222]:
pred_test =svc.predict(X_test_vec)
print(pred_test)

[0 1 1 ... 1 0 1]


In [223]:
subm = pd.read_csv("./dataset/sample_submission.csv")
subm['label'] = pred_test
subm

,id,label
0,1,0
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
4995,4996,0
4996,4997,0
4997,4998,1
4998,4999,0


In [224]:
subm.to_csv("submission.csv", index=False)